In [1]:
import requests
import pandas as pd
import pydantic
import datetime
from pathlib import Path
from collections import defaultdict

from BMRS_functions import BMRS_request
from BMRS_functions import get_BMRS_data

In [2]:
report = 'B1620'
date = '2020-01-01'
data = BMRS_request(report, date)
data.head()

,*DocumentType,BusinessType,ProcessType,TimeSeriesID,Quantity,CurveType,Resolution,SettlementDate,SettlementPeriod,PowerSystemResourceType,ActiveFlag,DocumentID,DocumentRevNum
0,Actual generation per type,Production,Realised,NGET-EMFIP-AGPT-TS-00950877,1733.0,Sequential fixed size block,PT30M,2020-01-01,48.0,Biomass,Y,NGET-EMFIP-AGPT-00166476,1.0
1,Actual generation per type,Production,Realised,NGET-EMFIP-AGPT-TS-00950876,0.0,Sequential fixed size block,PT30M,2020-01-01,48.0,Hydro Pumped Storage,Y,NGET-EMFIP-AGPT-00166476,1.0
2,Actual generation per type,Production,Realised,NGET-EMFIP-AGPT-TS-00950875,419.0,Sequential fixed size block,PT30M,2020-01-01,48.0,Hydro Run-of-river and poundage,Y,NGET-EMFIP-AGPT-00166476,1.0
3,Actual generation per type,Production,Realised,NGET-EMFIP-AGPT-TS-00950874,671.0,Sequential fixed size block,PT30M,2020-01-01,48.0,Fossil Hard coal,Y,NGET-EMFIP-AGPT-00166476,1.0
4,Actual generation per type,Production,Realised,NGET-EMFIP-AGPT-TS-00950873,4731.0,Sequential fixed size block,PT30M,2020-01-01,48.0,Fossil Gas,Y,NGET-EMFIP-AGPT-00166476,1.0


In [3]:
start_date = '2020-01-01'
end_date = '2020-01-05'

get_BMRS_data(report, start_date, end_date)

,*DocumentType,BusinessType,ProcessType,TimeSeriesID,Quantity,CurveType,Resolution,SettlementDate,SettlementPeriod,PowerSystemResourceType,ActiveFlag,DocumentID,DocumentRevNum
0,Actual generation per type,Production,Realised,NGET-EMFIP-AGPT-TS-00950877,1733.000,Sequential fixed size block,PT30M,2020-01-01,48.0,Biomass,Y,NGET-EMFIP-AGPT-00166476,1.0
1,Actual generation per type,Production,Realised,NGET-EMFIP-AGPT-TS-00950876,0.000,Sequential fixed size block,PT30M,2020-01-01,48.0,Hydro Pumped Storage,Y,NGET-EMFIP-AGPT-00166476,1.0
2,Actual generation per type,Production,Realised,NGET-EMFIP-AGPT-TS-00950875,419.000,Sequential fixed size block,PT30M,2020-01-01,48.0,Hydro Run-of-river and poundage,Y,NGET-EMFIP-AGPT-00166476,1.0
3,Actual generation per type,Production,Realised,NGET-EMFIP-AGPT-TS-00950874,671.000,Sequential fixed size block,PT30M,2020-01-01,48.0,Fossil Hard coal,Y,NGET-EMFIP-AGPT-00166476,1.0
4,Actual generation per type,Production,Realised,NGET-EMFIP-AGPT-TS-00950873,4731.000,Sequential fixed size block,PT30M,2020-01-01,48.0,Fossil Gas,Y,NGET-EMFIP-AGPT-00166476,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2635,Actual generation per type,Production,Realised,NGET-EMFIP-AGPT-TS-00952466,6636.000,Sequential fixed size block,PT30M,2020-01-05,1.0,Nuclear,Y,NGET-EMFIP-AGPT-00166621,1.0
2636,Actual generation per type,Production,Realised,NGET-EMFIP-AGPT-TS-00952465,136.000,Sequential fixed size block,PT30M,2020-01-05,1.0,Other,Y,NGET-EMFIP-AGPT-00166621,1.0
2637,Actual generation per type,Wind generation,Realised,NGET-EMFIP-AGPT-TS-00952464,5297.086,Sequential fixed size block,PT30M,2020-01-05,1.0,Wind Onshore,Y,NGET-EMFIP-AGPT-00166621,1.0
2638,Actual generation per type,Wind generation,Realised,NGET-EMFIP-AGPT-TS-00952463,2877.522,Sequential fixed size block,PT30M,2020-01-05,1.0,Wind Offshore,Y,NGET-EMFIP-AGPT-00166621,1.0
